In [167]:
cd ~/Documents/Galvanize/mtg-capstone/src/

/Users/benjaminwalzer/Documents/Galvanize/mtg-capstone/src


In [168]:
import pyspark as ps
from pyspark.mllib.recommendation import ALS
from pyspark.sql.types import StructField, StructType, IntegerType
import psycopg2
import os
import pandas as pd
import numpy as np
import datetime
from deck_scraping import ReflexiveDict

In [3]:
dbname = os.environ['CAPSTONE_DB_DBNAME']
host = os.environ['CAPSTONE_DB_HOST']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

In [129]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
cursor = conn.cursor()

In [29]:
# query db to get (deck_id, cardstorm_id, card_count) pairs
cursor.execute('''SELECT deck_id, cardstorm_id, card_count
                  FROM decks''')

In [30]:
# create schema for spark decks df
decks_schema = StructType([StructField('deck_id', IntegerType()),
                           StructField('cardstorm_id', IntegerType()),
                           StructField('card_count', IntegerType())])

# create spark df for decks
decks_df = spark.createDataFrame(data=cursor.fetchall(), schema=decks_schema)

In [31]:
# get a list of all unused cardstorm ids
cursor.execute('''SELECT cardstorm_id
                  FROM cards
                  WHERE cardstorm_id NOT IN (SELECT DISTINCT cardstorm_id FROM decks)''')

unused_ids = [_[0] for _ in cursor.fetchall()]

In [32]:
# create fake data for all unused cards
# deck_id of -1 is used to easily identify fake data
unused_cards = []
for unused_id in unused_ids:
    unused_cards.append((-1, unused_id, 1))

In [33]:
# make a new spark df from unused cards
unused_df = spark.createDataFrame(data=unused_cards, schema=decks_schema)

In [34]:
# merge both dataframes into one
complete_decks_df = decks_df.union(unused_df)

In [37]:
# create and train the ALS model
# using implicit train 
# rank is 10 for now, needs to be turned later
model = ALS.trainImplicit(ratings=complete_decks_df, rank=10)

In [38]:
# get the product features matrix out of the model (V)
product_features = model.productFeatures()

In [116]:
# turn the spark RDD into a spark DF, then into a pandas DF
pd_product_features = product_features.toDF().toPandas()

In [117]:
# create a cardstorm_id column, to be used as the index
pd_product_features['cardstorm_id'] = pd_product_features['_1']
pd_product_features['features'] = pd_product_features['_2']

# drop columns '_1' and '_2'
pd_product_features.drop(columns=['_1', '_2'], inplace=True)

# set the cardstorm_id to be the index
pd_product_features.set_index(keys='cardstorm_id', inplace=True, verify_integrity=True)

# sort the df by the index
pd_product_features.sort_index(inplace=True)

In [141]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
cursor = conn.cursor()

In [ ]:
# insert pd_product_features into db
current_date = str(datetime.date.today())
success = True
for cardstorm_id, features in pd_product_features.iterrows():
    if success == False:
        conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
        cursor = conn.cursor()
    query = 'INSERT INTO product_matrices (cardstorm_id, features, date) VALUES (%s, %s, %s)'
    
    try:
        cursor.execute(query, vars=[cardstorm_id, features.values[0], current_date])
    except psycopg2.IntegrityError:
        print('error: duplicate key found when inserting into features_matrices')
        success = False

In [143]:
conn.commit()

In [147]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
cursor = conn.cursor()

In [183]:
cursor.execute('''SELECT cardstorm_id, features 
                  FROM product_matrices 
                  WHERE date = (SELECT MAX(date) FROM product_matrices)
                  ORDER BY cardstorm_id ASC''')

In [184]:
# expand each row
product_matrix = []
indices = []
for idx, row in cursor.fetchall():
    indices.append(idx)
    product_matrix.append(row)

product_matrix = np.array(product_matrix)

In [185]:
product_matrix.shape

(11348, 10)

In [186]:
card_dict = ReflexiveDict()
card_dict.get_cards()

True

In [187]:
card_dict[262]

'anafenza, the foremost'

In [191]:
pd.DataFrame(data=product_matrix, index=indices, )

,0,1,2,3,4,5,6,7,8,9
1,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
2,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
3,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
4,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
5,4.937970e-04,-0.001094,0.000091,-0.002304,-0.002027,0.000515,-9.794660e-05,-1.352850e-03,-0.000574,-1.681620e-03
6,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
7,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
8,7.256920e-07,-0.000002,0.000002,-0.000001,-0.000002,-0.000001,-6.814930e-07,1.345710e-07,-0.000001,9.344210e-08
9,-9.973720e-04,0.002448,-0.000953,-0.000749,-0.000652,-0.000897,9.311400e-04,-1.454460e-03,-0.000501,4.246060e-04
10,4.460110e-01,-0.132993,-0.122812,0.119210,-0.248362,0.110401,-1.452570e-01,2.590660e-01,0.165110,3.149910e-01


In [188]:
product_matrix

array([[  7.25692000e-07,  -1.56811000e-06,   2.00104000e-06, ...,
          1.34571000e-07,  -1.13234000e-06,   9.34421000e-08],
       [  7.25692000e-07,  -1.56811000e-06,   2.00104000e-06, ...,
          1.34571000e-07,  -1.13234000e-06,   9.34421000e-08],
       [  7.25692000e-07,  -1.56811000e-06,   2.00104000e-06, ...,
          1.34571000e-07,  -1.13234000e-06,   9.34421000e-08],
       ..., 
       [  7.25692000e-07,  -1.56811000e-06,   2.00104000e-06, ...,
          1.34571000e-07,  -1.13234000e-06,   9.34421000e-08],
       [  7.25692000e-07,  -1.56811000e-06,   2.00104000e-06, ...,
          1.34571000e-07,  -1.13234000e-06,   9.34421000e-08],
       [ -1.46552000e-05,  -2.49798000e-04,   5.84726000e-04, ...,
         -2.21382000e-04,   1.13817000e-04,   3.42667000e-04]])

In [195]:
rep

'<pyspark.sql.session.SparkSession object at 0x109f466a0>'